# Spark DataFrames API


### Dataframe

- DataFrames are ditributed collections of records, all with pre-defined structure(schema - structure and data types of all columns)
-  DataFrames are built on Spark's core concepts but with structure, optimization and SQL-like operations for data manipulation.
- DataFrames track their schema and provide native support for many common SQL functions and relational operators
- DataFrames are evaluated as DAGs, using lazy evaluation and providing lineage and fault tolerance.
- DataFrames are immutable

### SparkContext vs SparkSession

- SparkSession is Spark application entry point. 
- Introduced in spark 2.0 as a unified entry point for all contexts (formerly instantiated individually as SparkContext, SQLContext, HiveContext, StreamingContext)

<i>Note: In databricks it is automatically created for you as spark</i>

### DataFrame API Optimizations

- **Adaptive Query Execution:** Dynamic plan adjustments during runtime based on actual data characteristics and execution patterns.
- **In-Memory Columnar Storage(Tungsten):** In-Memory coloumnar format for all the DataFrames enabling efficient analytical query performance and reduced memory footprint.
- **Built-in Statistics** - Automatic statistics collection when saving to optimized formats (Parqurt, Delta in databricks) enables smarter query planning and execution.
- **Catalyst Optimizer:** Query optimization engine that coverts DataFrame operations into an optimized execution plan


<i>**Note** Databricks comes with a native vectorized query engine that accelerates query execution using photon engine</i>

**DataFrame Query Planning:** 

- When a DataFrame is evaluated, the driver creates an optimized execution plan through a series of transformations 
- Converts the logical plan into phycal execution that minimizes resource usage and execution time. (Unresolved LP -> analysed LP -> optimized LP -> Physical Plan)




### DataFrame Data Types

#### Primitive

**`pyspark.sql.types.DataType`**

- `ByteType`
- `ShortType`
- `IntegerType`
- `LongType`
- `FloatType`
- `DoubleType`
- `BooleanType`
- `StringType`
- `BinaryType`
- `TimestampType`
- `DateType`

#### complex data types

- `ArrayType`
- `MapType`
- `StructType`

### Common DataFrame API methods

#### Transformations

##### Narrow Transformations

- narrow transformations process data within each partition independetly, without needing to combine data from other partitions.
- faster and more efficient because they avoid data shuffling between partitions. 

1. `select()` : selecting specific rows
2. `filter()`: Applying a filter condition to rows. 
3. `map()`: Applying a function to each row. 
4. `union()`: Combining two DataFrames with identical schemas. 
5. `withColumn()`: Adding a new column based on existing ones. 
6. `drop()`: Removing a column. 

##### Wide Transformations

- Wide transformations require data to be redistributed across partitions, often involving shuffling data based on keys.

1. `groupBy()`: Grouping data based on a column, which often requires shuffling to aggregate data from different partitions. 
2. `join()`: Joining two DataFrames, which requires shuffling data to combine rows based on a join key. 
3. `distinct()`: Removing duplicate rows, which might require shuffling to compare rows across partitions. 

#### Actions

1. `count()`: returns number of rows in a Dataframe
2. `show()`: display DataFrame content
3. `take(n)`: return first n rows from a DataFrame
4. `first()`: return first row from a DataFrame
5. `write()`: save DataFrame to storage

%md
#### Handiling missing values

**common functions**

- `isNull()`/`isNotNull()` - checks if values are null
- `count(col)` - counts non null values in a specific column
- `df.fillna()`/`df.na.fill()` - replace nulls with values
- `df.dropna()`/`df.na.drop()` - remove rows with nulls

#### referencing columns

- direct - `df.select("col_name")` - basic column selection

- by attribute - `df.select(df.attribute)` - column names that are valid python idenfiers, can be referenced across DataFrames(e.g., join)

- column expression - `df.select(df["col_name"])` - any column names, can be referenced across DataFrames

- column object - `df.select(col("name"))` - required when building complex expressions or using column specific operations like `cast()`, `alias()`, `asc()`or `desc()`


#### common column object methods

- `alias()` - rename column
- `cast()` - chnage data type
- `isNull()` or `isNotNull()` - check for nulls
- `contains()` - string matching
- `asc()`/`desc()` - sort direction - `df.sort(col("c1").asc())`


[pyspark.sql.Column](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.Column.html)

#### common built-in functions

- `concat(col1, col2)` - concatenate strings
- `date_format(col, fmt)` - fromat date strings
- `round(col, scale)` - round number to scale
- `regexp_replace(col, pattern, replace)` - replace using regex
- `coalesce(col1, col2)` - first non null value


[pyspark built in functions](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/functions.html)

#### user defined functions

- allows to use python functions on dataframe columns
- helpful to create custom reusable functions
- can impact performance as they can not be optimized by the Catalyst optimizer and has serialiation overhead.

```py
@udf("data_type")
def function_name(name):
    return value

df.select(function_name("name"))
```

<i><b>note:</b> pandas udfs are efficient becuase they operate on batches of raows instead of single rows, they leverage Apache Arrow for more efficient python-JVM serialization </i>

### Aggregate functions

can be applied in `agg()` method after `groupBy()` operation or directly within `select()`

- `sum()`
- `avg()`
- `min()`
- `max()`
- `count()`
- `first()`
- `last()`

### Spark SQL

- Spark SQL is a module in spark that allows you to run SQL queries on structured and semi-structured data.
- It provides a SQL-like interface on top of Spark's powerful DataFrame API, enabling both SQL and programmatic access to data, all with the same optimized execution engine.

- `createOrReplaceTempView()` : created a temp in-memory view that you can query with SQL.
- `createGlobalTempView()` : Creates a global temporary view accessible across sessions (under global_temp database).